<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
     <h3> Objective</h3> <br>
    <b style = "color:#000000"><i>Check and Rotate Expiring Access Keys for all IAM Users </i></b>
</div>
<br>
</center>

<center><h2><u>AWS Access Key Rotation</u></h2></center>

# Steps Overview
1)[ List All IAM Users](#1)<br>
2)[ List all Expiring Access Key](#2)<br>
3)[ Create AWS Access Key](#3)<br>
4)[ Update AWS Access Key](#4)<br>
5)[ Delete AWS Access Key](#5)<br>

<h3><a id='1'>List all IAM Users</a></h3>
This action lists all IAM Users for an AWS account.

>Action takes the following parameters: `None`<br>
>Action gives the following output: `all_users`

In [10]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_iam_users_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_list_all_iam_users(handle) -> List:
    """aws_list_all_iam_users lists all the IAM users

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of all IAM users
    """
    client = handle.client('iam')
    users_list=[]
    response = client.list_users()
    try:
        for x in response['Users']:
            users_list.append(x['UserName'])
    except Exception as e:
        users_list.append(e)
    return users_list


task = Task(Workflow())
task.configure(outputName="all_users")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_iam_users, lego_printer=aws_list_all_iam_users_printer, hdl=hdl, args=args)

<h3><a id='2'>List all Expiring Access Keys</a></h3>
Using unSkript's AWS List Expiring Access Keys action we will list those users whose Access Keys past the given threshold number of days i.e. expiring.

>Action takes the following parameters: `threshold_days`

>Action captures the following output: `all_expiring_users`

In [13]:
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import dateutil
from pydantic import BaseModel, Field
from typing import Dict,List
import pprint
import datetime

from beartype import beartype
@beartype
def aws_list_expiring_access_keys_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_list_expiring_access_keys(handle, threshold_days: int, aws_username: str='')-> Dict:
    """aws_list_expiring_access_keys returns all the ACM issued certificates which are about to expire given a threshold number of days

        :type handle: object
        :param handle: Object returned from Task Validate

        :type threshold_days: int
        :param threshold_days: Threshold number of days to check for expiry. Eg: 30 -lists all access Keys which are expiring within 30 days

        :type aws_username: str
        :param aws_username: Username of the IAM User

        :rtype: Result Dictionary of result
    """
    try:
        iamClient = handle.client('iam')
        response = iamClient.list_access_keys(UserName=aws_username)
        result = {}
        for x in response["AccessKeyMetadata"]:
            if len(response["AccessKeyMetadata"])!= 0:
                create_date = x["CreateDate"]
                right_now = datetime.datetime.now(dateutil.tz.tzlocal())
                diff = right_now-create_date
                days_remaining = diff.days
                if days_remaining > threshold_days:
                    result["username"] = x["UserName"]
                    result["access_key_id"] = x["AccessKeyId"]
    except Exception as e:
        result["error"] = e
    return result


task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "aws_username": "iter_item",
    "threshold_days": "threshold_days"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_users",
    "iter_parameter": "aws_username"
    }''')
task.configure(outputName="all_expiring_users")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_expiring_access_keys, lego_printer=aws_list_expiring_access_keys_printer, hdl=hdl, args=args)

<h3>List of Expiring Users</h3>
This action lists the usernames of expiring Access Keys using the output from Step 2.

In [14]:
def expiring_users_list():
    res=[]
    for k,v in all_expiring_users.items():
        if len(v)!=0:
            res.append(k)
    print(res)
    return res
expiring_users = expiring_users_list()

task.configure(outputName="expiring_users")

<h3><a id='3'>List of Expiring Users and Access Keys</a></h3>
This action simply creates another list containing a dictionary of the user and their old access key. The output from this acion is required for Step 4 and Step 5.

In [7]:
def expired_access_keys_list():
    res=[]
    for k,v in all_expiring_users.items():
        if len(v)!=0:
            res.append(v)
    print(res)
    return res
expiring_access_key_list = expired_access_keys_list()

task.configure(outputName="expiring_access_key_list")

<h3><a id='3'>Create AWS Access Keys</a></h3>
Using unSkript's AWS Create Access Key action we will create a new Access Key for the users from Step 2. 

>Action takes the following parameters: `aws_username`


In [8]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict, List
import pprint


from beartype import beartype
@beartype
def aws_create_access_key_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_create_access_key(
    handle,
    aws_username: str
) -> Dict:
    """aws_create_access_key creates a new access key for the given user.
        :type handle: object
        :param handle: Object returned from Task Validate

        :type aws_username: str
        :param aws_username: Username of the IAM user to be looked up

        :rtype: Result Dictionary of result
    """
    iamClient = handle.client('iam')
    result = iamClient.create_access_key(UserName=aws_username)
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "aws_username": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "expiring_users",
    "iter_parameter": "aws_username"
    }''')
task.configure(outputName="new_access_keys")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_access_key, lego_printer=aws_create_access_key_printer, hdl=hdl, args=args)

<h3><a id='4'>Update AWS Access Key</a></h3>
Using the AWS Update Access Key action we will update the status of the old Access Key to <b>"Inactive"</b>. This step is required to delete the old access key as one user cannot have two Access Keys. 

>This action takes the following parameters: `aws_username`, `aws_access_key_id` and `status`

In [10]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict,List
from unskript.enums.aws_access_key_enums import AccessKeyStatus
import pprint


from beartype import beartype
@beartype
def aws_update_access_key_printer(output):
    if output is None:
        return
    pprint.pprint("Access Key status successfully changed")
    pprint.pprint(output)


@beartype
def aws_update_access_key(
    handle,
    aws_username: str,
    aws_access_key_id: str,
    status: AccessKeyStatus
) -> Dict:
    """aws_update_access_key updates the status of an access key to Inactive/Active
        :type handle: object
        :param handle: Object returned from Task Validate

        :type aws_username: str
        :param aws_username: Username of the IAM user to be looked up

        :type aws_access_key_id: str
        :param aws_access_key_id: Old Access Key ID of the user of which the status needs to be updated

        :type status: AccessKeyStatus
        :param status: Status to set for the Access Key

        :rtype: Result Dictionary of result
    """
    iamClient = handle.client('iam')
    result = iamClient.update_access_key(UserName=aws_username, AccessKeyId=aws_access_key_id, Status=status)
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "aws_access_key_id": "iter.get(\\"access_key_id\\")",
    "aws_username": "iter.get(\\"username\\")",
    "status": "AccessKeyStatus.Inactive"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "expiring_access_key_list",
    "iter_parameter": ["aws_username","aws_access_key_id"]
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_update_access_key, lego_printer=aws_update_access_key_printer, hdl=hdl, args=args)

<h3><a id='5'>Delete AWS Access Key</a></h3>
Finally, we will delete the the old (Inactive) Access Key for the IAM Users

>This action takes the following parameters: `aws_username` and `aws_access_key_id`

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
from typing import Dict,List
import pprint


from beartype import beartype
@beartype
def aws_delete_access_key_printer(output):
    if output is None:
        return
    pprint.pprint("Access Key successfully deleted")
    pprint.pprint(output)


@beartype
def aws_delete_access_key(
    handle,
    aws_username: str,
    aws_access_key_id: str,
) -> Dict:
    """aws_delete_access_key deleted the given access key.
        :type handle: object
        :param handle: Object returned from Task Validate

        :type aws_username: str
        :param aws_username: Username of the IAM user to be looked up

        :type aws_access_key_id: str
        :param aws_access_key_id: Old Access Key ID of the user which needs to be deleted

        :rtype: Result Status Dictionary of result
    """
    iamClient = handle.client('iam')
    result = iamClient.delete_access_key(UserName=aws_username, AccessKeyId=aws_access_key_id)
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "aws_access_key_id": "iter.get(\\"access_key_id\\")",
    "aws_username": "iter.get(\\"username\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "expiring_access_key_list",
    "iter_parameter": ["aws_access_key_id","aws_username"]
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_access_key, lego_printer=aws_delete_access_key_printer, hdl=hdl, args=args)

<h3>Conclusion</h3>
<p>In this Runbook, we were able to perform AWS Access Key rotation for IAM users whose Access Keys were expiring by using unSkript's AWS actions. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io">us.app.unskript.io</a></p>